<a href="https://colab.research.google.com/github/yuanchaohu/Colab_learning/blob/main/NLP_pytorch_03_01b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 29.8 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [1]:
import torch as tc
from torchtext.legacy import data, datasets
import random

In [2]:
seed = 42
tc.manual_seed(seed)

device = tc.device('cuda' if tc.cuda.is_available() else 'cpu')
print(device)

cuda


**Fields**

[Check documentation](https://pytorch.org/text/_modules/torchtext/data/field.html)

In [3]:
# define fields
TEXT = data.Field(tokenize='spacy', lower=True)

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [4]:
TEXT

In [5]:
LABEL = data.LabelField()
LABEL

**Text REtrieval Conference (TREC) Question Classification Dataset**

*Data Examples and Six Categories:*

| Text | Label | Category |
| --- | --- | --- |
|CNN is the abbreviation for what ?|ABBR| ABBREVIATION |
| What is the date of Boxing Day ? | NUM |NUMERIC|
|Who discovered electricity ?| HUM |HUMAN|
|What 's the colored part of the eye called ?|ENTY|ENTITY|
|Why do horseshoes bring luck ?|DESC|DESCRIPTION|
|What is California 's capital ?|LOC|LOCATION|

In [6]:
train, test = datasets.TREC.splits(TEXT, LABEL)
train, val = train.split(random_state=random.seed(seed))

downloading train_5500.label


train_5500.label: 100%|██████████| 336k/336k [00:00<00:00, 13.4MB/s]


downloading TREC_10.label


TREC_10.label: 100%|██████████| 23.4k/23.4k [00:00<00:00, 27.8MB/s]


In [7]:
vars(train[-1])

{'text': ['in',
  'order',
  'from',
  'the',
  'top',
  ',',
  'the',
  'four',
  'stripes',
  'on',
  'a',
  'can',
  'of',
  'pepsi',
  'are',
  'what',
  'colors',
  '?'],
 'label': 'ENTY'}

In [8]:
# build vocab
TEXT.build_vocab(train, min_freq=2)
LABEL.build_vocab(train)

In [9]:
print(LABEL.vocab.stoi)

defaultdict(None, {'HUM': 0, 'ENTY': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


In [10]:
print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

Vocabulary size of TEXT: 2684
Vocabulary size of LABEL: 6


In [11]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, val, test),
    batch_size = 64,
    sort_key=lambda x: len(x.text), 
    device=device
)